# An Introduction to GRASS GIS for Tangible Landscape

***Caitlin Haedrich, Pratikshya Regmi, Anna Petrasova and Helena Mitasova***

*Center for Geospatial Analytics at NC State University*

In this notebook, we will become familiar with the GRASS working environment and toolsets. By the end of this notebook, you will have experience with:

* [Creating a new GRASS project](#2.-Create-a-New-Project)
* [Importing data](#4.-Import-Data)
* [Working with GRASS Tools](#5.-GRASS-GIS-Tools) and [the Python API](#6.-GRASS-Python-API)
* [Visualizing data](#7.-Data-Visualization-with-grass.jupyter)

Let's dive in!


***

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

We are going to import the GRASS GIS Python API (`grass.script`) and the GRASS GIS Jupyter package (`grass.jupyter`), but first, we'll need to ask `grass` to check it's `--config` to see where the python packages are then add them to the system path before we can import them. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

***

## 2. Create a New Project

Projects are defined by a Coordinate Reference System (CRS). We can set the CRS from a georeferenced file (such as a Geotiff) or an EPSG string. Here, we use [EPSG 3358](https://epsg.io/3358), a projection for NC in meters.

In [ ]:
gs.create_project("nc-swine", epsg=3358, overwrite=True)

We could also create a project from a georeferenced file, such as `lagoons.gpkg` which we will use later in this workshop.

In [ ]:
# gs.create_project("nc-swine", filename="lagoons.gpkg", overwrite=True)

***

## 3. Start GRASS Session

In [ ]:
gj.init("./nc-swine/PERMANENT");

We've launched GRASS GIS now! We can access GRASS GIS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.list type=all

In [ ]:
!g.region -p

---

## 4. Import Data

In [ ]:
!v.import input="./lagoons.gpkg" output="lagoons"

In [ ]:
!g.region -a vector="lagoons" res=10

In [ ]:
!g.region grow=200 -p

In [ ]:
!g.list type=all

We're going to import a digital elevation model (DEM), we will use a GRASS addon [r.in.usgs](https://grass.osgeo.org/grass-devel/manuals/addons/r.in.usgs.html), which uses [TNM Access](https://apps.nationalmap.gov/tnmaccess/) REST API to access USGS data. First install the addon:

In [ ]:
!g.extension r.in.usgs

Download and reproject a 1/9 arc-second DEM (approx 3-m resolution):

In [ ]:
!r.in.usgs product="ned" ned_dataset="ned19sec" output_name="elevation"

<details>

<summary>Alternative Import Method</summary>

### Download with wget and import with `r.import`

First, download and unzip with bash.

```bash
%%bash
wget https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/19/IMG/ned19_n35x00_w078x00_nc_statewide_2003.zip
unzip ned19_n35x00_w078x00_nc_statewide_2003.zip
```

We'll import our elevation model using [`r.import`](https://grass.osgeo.org/grass-devel/manuals/r.import.html) and create a raster layer called "elevation". The `r.import` tool will reproject the data to the project CRS (thereby avoiding any future CRS mismatches - nice!). We also set it to only import the area within the computational region and to resample it using bilinear interpolation to the resolution of the computational region.

```bash
!r.import input="ned19_n35x00_w078x00_nc_statewide_2003.img" output="elevation" resample="bilinear" extent="region"
```

</details>

In [ ]:
!g.list type=all

In [ ]:
!r.in.wms url="https://imagery.nationalmap.gov/arcgis/services/USGSNAIPPlus/ImageServer/WMSServer" out="ortho" layer="USGSNAIPPlus"

***

## 5. GRASS GIS Tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools or extensions that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

Check out the _brand new_ [manual page](https://grass.osgeo.org/grass-devel/manuals/full_index.html) to browse tools.

There is also a tool for finding other tools:

In [ ]:
!g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool through command line:

In [ ]:
!r.univar --help

***

## 6. GRASS Python API

There are two Python APIs for accessing GRASS GIS tools' functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows and low level tasks. Here, we will be using the Python Scripting Library (`import grass.script as gs`)
as it is simpler and more straightforward to use.
 

The GRASS GIS Python Scripting Library provides functions to call GRASS tools within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file

Here's an example of the Python API in action:

In [ ]:
gs.run_command("g.list", type="raster")

**Try it yourself!**

_The `r.info map=elevation` command will print information about the elevation raster. Execute `r.info` in using the Python API._

<details>
    <summary>👉 <b>click to see solution</b></summary>
    
```python
gs.read_command("r.info", map="elevation")
```
</details>

The Python API also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

_Try using `gs.vector_info` to print information about the "lagoons" vector layer._

<details>
    <summary>👉 <b>click to see solution</b></summary>
    
```python
gs.vector_info("lagoons")
```
</details>

***

## 7. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="elevation") # d.rast map=elevation
example.d_barscale(bgcolor="none") # d.barscale
example.d_legend(raster="elevation") #d.legend
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

`grass.jupyter.InteractiveMap()` creates leaflet maps that are interactive. They can also be saved as html with the `save()` method and embedded on a website or shared.

In [ ]:
map = gj.InteractiveMap()
map.add_raster("ortho", opacity=0.6)
map.add_vector("lagoons")
map.add_layer_control()
map.show()

**Try it yourself!**

_Change the following code so that the ortho photo is displayed over the shaded relief instead of elevation._

In [ ]:
gs.run_command("r.relief", input="elevation", output="relief")

In [ ]:
m = gj.Map()
m.d_shade(color="elevation", shade="relief", brighten=50) # d.shade
m.d_vect(map="lagoons", color="none", fill_color="blue")
m.d_barscale(bgcolor="none") # d.barscale
m.show()

<details>
    <summary>👉 <b>click to see solution</b></summary>
    
```python
m.d_shade(color="ortho", shade="relief", brighten=50) 
```
</details>